In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
from jc3000 import Sequence

In [ ]:
#!pip install jc3000

Note: you may need to restart the kernel to use updated packages.


In [7]:
#notes dictionary
seq2note_dict= {
    "000": "c",
    "001": "d",
    "010": "e",
    "011": "f",
    "100": "g",
    "101": "a",
    "110": "b",
    "111": "C"    
}

#probability idictionary
prob_next_one = {
    "000": 0.8,
    "001": 0.6,
    "010": 0.5,
    "011": 0.3,
    "100": 0.7,
    "101": 0.4,
    "110": 0.8,
    "111": 0.1
    
}

prob_next_zero = {key: round(1-(value),1) for key, value in prob_next_one.items()}

print(prob_next_zero)
print(seq2note_dict["001"])

{'000': 0.2, '001': 0.4, '010': 0.5, '011': 0.7, '100': 0.3, '101': 0.6, '110': 0.2, '111': 0.9}
d


In [8]:
#generate a sequence of length n
def generate_sequence(n):

    #generate first state = 3 random binary digits
    s1=[]
    for i in range(3):
        s1.append(random.randint(0,1))

    #create a list of sequences
    prob_1_seq = []
    prob_0_seq = []
    notes_seq = []
    notes_string_seq = []


    for i in range (n-1):
        numbers = ''.join(str(e) for e in s1) #convert list to string
        last_3 = numbers[-3:] #get last 3 digits

        prob_1 = prob_next_one[last_3] # retrieve the probability of the next number being one
        prob_0 = prob_next_zero[last_3]
        prob_1_seq.append(prob_1)
        prob_0_seq.append(prob_0)


        #generate 1 or 0 based on the probability distribution defined in the dictionary
        next_num= np.random.binomial(size=1, n=1, p=prob_1)
        next_num= int(next_num)
        s1.append(next_num)

        #get note from sequence
        note = seq2note_dict[last_3]
        notes_seq.append(note)

    #get the probability of the last note
    last_note_prob = ''.join(str(e) for e in s1) # convert the finished sequence into a string
    last_triad = last_note_prob[-3:] #get the very last triad

    #get the last note
    last_note = seq2note_dict[last_triad]
    notes_seq.append(last_note)

    #last probability 
    last_prob1= prob_next_one[last_triad]
    last_prob0=prob_next_zero[last_triad]

    prob_1_seq.append(last_prob1)
    prob_0_seq.append(last_prob0)
    
    #just to have the triads
    triad_seq = []
    for i in range(len(s1)-2):
        binary_string= ''.join(str(e) for e in s1)
        triad = binary_string[i:i+3]
        triad_seq.append(triad)

    return s1, triad_seq, prob_1_seq, prob_0_seq, notes_seq
    
#print (s1)
#print(notes_seq)
#print(prob_1_seq)
#print(prob_0_seq)


    

In [9]:
(binary_seq, triad_seq, prob_1, prob_0, notes) = generate_sequence(8)

print(f"this is the sequence:{binary_seq}\nThe triads in the sequence:{triad_seq}\nThe probability that the next item in the sequence is 1:{prob_1}\nThe probability that the next item in the sequence is 0:{prob_0}\nAnd the notes:{notes}")

this is the sequence:[0, 0, 0, 1, 0, 0, 1, 0, 1, 0]
The triads in the sequence:['000', '001', '010', '100', '001', '010', '101', '010']
The probability that the next item in the sequence is 1:[0.8, 0.6, 0.5, 0.7, 0.6, 0.5, 0.4, 0.5]
The probability that the next item in the sequence is 0:[0.2, 0.4, 0.5, 0.3, 0.4, 0.5, 0.6, 0.5]
And the notes:['c', 'd', 'e', 'g', 'd', 'e', 'a', 'e']


In [10]:
#information content = -log(p)
def information_content(p):
    return -np.log(p)

In [62]:
def generate_sound_data(frequency, volume=100, waveform="sine", duration = 0.1,fs = 44100):
      # seconds
      # Sample rate
    t = np.linspace(0, duration, int(fs * duration), False)

    sound = np.zeros_like(t)

    if waveform == 'sine':
        sound = np.sin(frequency * t * 2 * np.pi)
    elif waveform == 'square':
        sound = np.sign(np.sin(frequency * t * 2 * np.pi))
    elif waveform == 'sawtooth':
        sound = 2 * (t * frequency % 1) - 1
    elif waveform == 'triangle':
        sound = 2 * np.abs(2 * (t * frequency - np.floor(0.5 + t * frequency))) - 1
    elif waveform == 'pulse wave':
        sound = np.where((t % (1 / frequency)) < (1 / frequency) * 0.5, 1.0, -1.0)  # 0.5 in this case is the duty cycle
    elif waveform == 'white noise':
        samples = int(fs * duration)
        sound = np.random.uniform(low=-1.0, high=1.0, size=samples)

    return sound * volume

for i in seq2note_dict:
    print (seq2note_dict[i], generate_sound_data(seq2note_dict[i]))

600 [  0.           8.53814343  17.01393003 ... -25.36545839 -17.01393003
  -8.53814343]
700 [  0.           9.95678466  19.81461432 ... -29.47551744 -19.81461432
  -9.95678466]
800 [  0.          11.37340476  22.59921073 ... -33.53173459 -22.59921073
 -11.37340476]
900 [  0.          12.78771617  25.36545839 ... -37.52670049 -25.36545839
 -12.78771617]
1000 [  0.          14.1994318   28.11111133 ... -41.45311767 -28.11111133
 -14.1994318 ]
1100 [  0.          15.60826508  30.83394031 ... -45.30381388 -30.83394031
 -15.60826508]
1200 [  0.          17.01393003  33.53173459 ... -49.0717552  -33.53173459
 -17.01393003]
1300 [  0.          18.41614132  36.20230379 ... -52.75005885 -36.20230379
 -18.41614132]


In [61]:
wave_arrays = []
for j in range(4):
    (binary_seq, triad_seq, prob_1, prob_0, notes) = generate_sequence(8)
    
    for k in notes: 
        data= generate_sound_data(k)
        #wave_arrays.append(data)
    print(data)
    


[  0.          11.37340476  22.59921073 ... -33.53173459 -22.59921073
 -11.37340476]
[  0.          17.01393003  33.53173459 ... -49.0717552  -33.53173459
 -17.01393003]
[  0.          17.01393003  33.53173459 ... -49.0717552  -33.53173459
 -17.01393003]
[  0.          12.78771617  25.36545839 ... -37.52670049 -25.36545839
 -12.78771617]


In [ ]:
#this generates n melodies of length l 
def get_n_melodies(n,l):
    tavola=[]
    tbl = pd.DataFrame(index=["binary_sequence", "triads", "melodies", "-log(p(1))", "-log(p(0))", "entropy", "wave_arrays"])
    
    for j in range(n):
        (binary_seq, triad_seq, prob_1, prob_0, notes) = generate_sequence(l)

        
        #add values to dataframe
        ic1= information_content(prob_1)
        ic1=[round(i,2) for i in ic1]
        ic0= information_content(prob_0)
        ic0=[round(i,2) for i in ic0]
        
        #H= p(1) * -log(p(1)) + p(0) * -log(p(0))
        entropy = prob_1 @  information_content(prob_1)  + prob_0 @ information_content(prob_0)
        entropy = round(entropy,2) 
        
        for k in notes: 
            wave_arrays= generate_sound_data(k)
        
        #put the data in the table
        tbl[f"melody{j}"] = [binary_seq, triad_seq, notes, ic1, ic0, entropy, wave_arrays]
        
        tavola.append(tbl)
        final = pd.concat(tavola, axis = 1)
        
        
        
        
        

In [11]:
ic1= information_content(prob_1)
ic1=[round(i,2) for i in ic1]
ic0= information_content(prob_0)
ic0=[round(i,2) for i in ic0]

entropy = prob_1 @  information_content(prob_1)  + prob_0 @ information_content(prob_0)
entropy = round(entropy,2) 

print(ic1)
print(ic0)
print(entropy)

[0.22, 0.51, 0.69, 0.36, 0.51, 0.69, 0.92, 0.69]
[1.61, 0.92, 0.69, 1.2, 0.92, 0.69, 0.51, 0.69]
5.21


In [ ]:
if __name__== "__main__":
    pass